## git에 업로드~

# 0.함수 정의 - 테이블에서 데이터 추출

In [5]:
def choochool(result_list,table):
    #테이블에서 행 추출 
    rows = table.find_elements_by_css_selector('tr')
    for row in rows:
    # 내용 없는 행 제거
        if len(row.text)>0:
            new_list = row.text.split(' ')
            #왜 try except을 쓰는가? elem이 있는 것도 있고 없는 것도 있는 친구에만 조건문 진행을 원함
            #그러니깐 그냥 오류 뜨면 무시하고 진행하고 오류 안 뜰 때 즉, 내가 원하는 거 찾을 때만 원하는 코드 실행되도록
            try:
                if row.find_element_by_css_selctor('td:nth-child(3) img').get_attribute('alt') =='하락':
                    new_list[2]='-'+new_list[2]
            except:
                pass
        result_list.append(new_list)
    return result_list

# 1.함수 정의 - 메인 크롤링(페이지 이동 및 테이블 크롤링, df 출력

In [20]:
def main_crawling():
    #시간 측정
    start = time.time()
    #페이지 이동
    last_page = driver.find_element_by_css_selector('td.pgRR > a').get_attribute('href').split('=')[-1]
    #빈 리스트 만들기
    result_list = []
    progress = 10
    for i in range(1,3):   #int(last_page)+1
        #버튼 선택하기 및 클릭하기
        page_button_list = driver.find_element_by_css_selector('tbody:nth-child(2)')
        page_button = page_button_list.find_element_by_link_text('{}'.format(i))
        page_button.click()
        #데이터 뽑을 테이블 요소 가져오기
        table = driver.find_element_by_css_selector('tbody')
        #리스트에 추가 - 함수 choochool활용
        result_list = choochool(result_list,table)
        if (i/int(last_page)) >= progress/100: 
            print(str(round(i/int(last_page)*100))+'% 진행 / 경과시간 :'+str(time.time()-start))
            progress +=10
        if i%10==0:
            page_button_list = driver.find_element_by_css_selector('tbody:nth-child(2)')
            page_button = page_button_list.find_element_by_link_text('다음')
            page_button.click()
    df = pd.DataFrame(result_list[1:],columns=result_list[0]).set_index('날짜')
    return df


# 2.최종 실행 코드

In [21]:
# 임포트
from selenium import webdriver
import pandas as pd
import time
#시간 쟤기
start = time.time()
#코드 입력 및 url 설정
url = 'https://finance.naver.com/item/sise.nhn?code='
code = input('코드? ')
final_url = url+code
#크롬 동작 및 url 이동
driver = webdriver.Chrome('chromedriver')
driver.get(final_url)
#iframe 내부로 이동
daily_table = driver.find_element_by_css_selector('#content > div.section.inner_sub > iframe:nth-child(4)')
driver.switch_to.frame(daily_table)

result = main_crawling()
print(str(time.time() - start)+'만큼 소요됐습니다')
print(result)

코드? 068270
10.258018493652344만큼 소요됐습니다
                 종가     전일비       시가       고가       저가        거래량
날짜                                                               
날짜               종가     전일비       시가       고가       저가        거래량
2021.04.30  268,000     500  270,000  273,000  265,000    374,643
2021.04.29  268,500   1,500  270,500  277,000  264,000  1,036,746
2021.04.28  270,000   4,000  272,500  282,000  266,000  1,371,738
2021.04.27  274,000  11,000  285,000  285,500  273,500  1,344,606
2021.04.26  285,000   3,000  288,000  288,500  284,000    716,990
2021.04.26  285,000   3,000  288,000  288,500  284,000    716,990
2021.04.26  285,000   3,000  288,000  288,500  284,000    716,990
2021.04.26  285,000   3,000  288,000  288,500  284,000    716,990
2021.04.23  288,000       0  288,000  291,000  285,000    521,531
2021.04.22  288,000   2,500  292,000  294,500  287,000    772,035
2021.04.21  290,500   9,500  295,500  297,500  290,000  1,004,575
2021.04.20  300,000   9,500  306,500 